In [1]:
import pandas as pd
import random as rnd 
import mysql.connector
from mysql.connector import Error
import plotly
import json
import csv
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import pymysql
import plotly.express as px
import plotly.graph_objects as go
from sqlalchemy import create_engine
import networkx as nx
from networkx_viewer import Viewer
from functools import reduce
from scipy.spatial.distance import pdist, squareform
import jgraph
from cyjupyter import Cytoscape
from scipy.stats import kendalltau
from scipy.stats import weightedtau

In [2]:
## Connection 

In [3]:
connection = mysql.connector.connect(host='127.0.0.1',
                                         database='employment',
                                         user='root',
                                         password='thanhn123')

In [4]:
def get_data(connection, descriptor_domain,filt = False,val = None):
    '''
    Return df in the SOCs-decriptor format of a specific domain 
    descriptor_domeain:
        work context
        knowledge
        skills
        gwas
        job_zones, filter
        Abilities, filter
    
    '''
    df= pd.DataFrame(data=None)
    if descriptor_domain == 'work_context':
        df= pd.read_sql('''
            SELECT title, element_name, data_value FROM employment.work_context
            LEFT JOIN employment.content_model_reference USING (element_id) 
            LEFT JOIN employment.occupation_data USING (onetsoc_code)
            WHERE scale_id LIKE 'CX';''', con = connection)
        df = df.reset_index().pivot(index = 'title', columns = 'element_name', values = 'data_value')
    elif descriptor_domain == 'job_zones':
        if filt:
            df= pd.read_sql('''
                SELECT title,job_zone FROM employment.job_zones
                LEFT JOIN employment.occupation_data USING (onetsoc_code); ''', con = connection)
            df = df[df['job_zone'].isin(val)]
        else:
            df= pd.read_sql('''
                SELECT title,job_zone FROM employment.job_zones
                LEFT JOIN employment.occupation_data USING (onetsoc_code);''', con = connection)
    elif descriptor_domain == 'abilities':
        if filt:
#             df= pd.read_sql('''
#             SELECT title, element_name, element_id,data_value FROM employment.''' + descriptor_domain + '''
#             LEFT JOIN employment.content_model_reference USING (element_id) 
#             LEFT JOIN employment.occupation_data USING (onetsoc_code); ''', con = connection)
#             df = df[df['element_id'].contains(val)]
            df= pd.read_sql('''
            SELECT title, element_name, data_value FROM employment.''' + descriptor_domain + '''
            LEFT JOIN employment.content_model_reference USING (element_id) 
            LEFT JOIN employment.occupation_data USING (onetsoc_code)
            WHERE element_id LIKE\''''+ val +'''%\'; ''', con = connection)
            
        else:
            df= pd.read_sql('''
            SELECT title, element_name,element_id, data_value FROM employment.''' + descriptor_domain + '''
            LEFT JOIN employment.content_model_reference USING (element_id) 
            LEFT JOIN employment.occupation_data USING (onetsoc_code);''', con = connection)
        df = df.groupby(by=['title','element_name'])['data_value'].mean()
        df = df.reset_index().pivot(index = 'title', columns = 'element_name', values = 'data_value')
    elif descriptor_domain == 'work_values':
        df= pd.read_sql('''
            SELECT title, element_name, data_value FROM employment.work_values
            LEFT JOIN employment.content_model_reference USING (element_id) 
            LEFT JOIN employment.occupation_data USING (onetsoc_code)
            WHERE scale_id LIKE 'EX';''', con = connection)
        df = df.reset_index().pivot(index = 'title', columns = 'element_name', values = 'data_value')
    elif descriptor_domain == 'interests':
        df= pd.read_sql('''
            SELECT title, element_name, data_value FROM employment.interests
            LEFT JOIN employment.content_model_reference USING (element_id) 
            LEFT JOIN employment.occupation_data USING (onetsoc_code)
            WHERE scale_id LIKE 'OI';''', con = connection)
        df = df.reset_index().pivot(index = 'title', columns = 'element_name', values = 'data_value')
    elif descriptor_domain == 'work_styles':
        df= pd.read_sql('''
            SELECT title, element_name, data_value FROM employment.work_styles
            LEFT JOIN employment.content_model_reference USING (element_id) 
            LEFT JOIN employment.occupation_data USING (onetsoc_code)
            WHERE scale_id LIKE 'IM';''', con = connection)
        df = df.reset_index().pivot(index = 'title', columns = 'element_name', values = 'data_value')
    elif descriptor_domain == 'name':
        df= pd.read_sql('''
            SELECT * FROM employment.occupation_data;''', con = connection)
    else:
        df= pd.read_sql('''
            SELECT title, element_name, data_value FROM employment.''' + descriptor_domain + '''
            LEFT JOIN employment.content_model_reference USING (element_id) 
            LEFT JOIN employment.occupation_data USING (onetsoc_code);''', con = connection)
        df = df.groupby(by=['title','element_name'])['data_value'].mean()
        df = df.reset_index().pivot(index = 'title', columns = 'element_name', values = 'data_value')
    return df

In [5]:
df_cps = pd.read_csv('../CPS Job Changes/JobChanges_2011to19.csv')

In [6]:
df_cps.sort_values(by = 'pct_tot', ascending = False)

,ONET18_SOC_LY,ONET18_SOC,ONET18_Title,ONET18_Title_LY,count,pct_tot,count_socly,pct_socly,in_grp,in_minor_grp,in_broad_occ,employment_projection_NewJob,wage_change_BetweenJobs
75823,41-1011,41-1012,First-Line Supervisors of Non-Retail Sales Wor...,First-Line Supervisors of Retail Sales Workers,734264.510000,4.395486e-01,2207170.940,33.267224,1,1,1,-8.3,85.257732
76141,41-1012,41-1011,First-Line Supervisors of Retail Sales Workers,First-Line Supervisors of Non-Retail Sales Wor...,603472.900000,3.612536e-01,999832.010,60.357429,1,1,1,-5.5,-46.021146
77141,41-2012,41-2031,Retail Salespersons,Gambling Change Persons and Booth Cashiers,509346.270000,3.049071e-01,2630467.290,19.363338,1,1,0,-0.6,-1.700405
76588,41-2011,41-2031,Retail Salespersons,Cashiers,509346.270000,3.049071e-01,2630467.290,19.363338,1,1,0,-0.6,6.772208
177,11-1021,11-1011,Chief Executives,General and Operations Managers,352402.430000,2.109567e-01,751730.590,46.878820,1,1,0,-10.0,83.034056
...,...,...,...,...,...,...,...,...,...,...,...,...,...
88371,45-2092,25-1112,"Law Teachers, Postsecondary","Farmworkers and Laborers, Crop, Nursery, and G...",1.135947,6.800058e-07,184416.524,0.000616,0,0,0,6.9,346.293792
88369,45-2092,25-1082,"Library Science Teachers, Postsecondary","Farmworkers and Laborers, Crop, Nursery, and G...",1.135947,6.800058e-07,184416.524,0.000616,0,0,0,3.0,180.717341
88368,45-2092,25-1081,"Education Teachers, Postsecondary","Farmworkers and Laborers, Crop, Nursery, and G...",1.135947,6.800058e-07,184416.524,0.000616,0,0,0,4.8,157.524058
88367,45-2092,25-1072,"Nursing Instructors and Teachers, Postsecondary","Farmworkers and Laborers, Crop, Nursery, and G...",1.135947,6.800058e-07,184416.524,0.000616,0,0,0,17.6,193.257438


## ONET 

In [7]:
def distance_table(df, jz=False):
    
#     squareform(pdist(df.iloc[:, 1:])), columns=df.title.unique(), index=df.title.unique()
    dist = []
    name_dict = {}
    if jz:
        name_dict = df['title'].sort_values().reset_index()['title'].to_dict()
        dist = pdist(df[df.columns[1:]],'euclidean')
    else:
        for i in range(len(df.index)):
            name_dict[i] = df.index[i]
        dist = pdist(df,'euclidean') #Euclidean distance for 1XN
    df_dist = pd.DataFrame(squareform(dist)) #Euclidean distance for NxN
#     return df_dist
    columns = list(df_dist.index) 
    for i in range(len(df.index)): #convert 0 values to NaN
        df_dist.loc[i,i] = np.NaN
    mean = df_dist.mean().mean()
    std = df_dist.stack().std() 

    for col in columns: #Z-score calculated 
        df_dist[col] = (df_dist[col] - mean)/std
#     return df_dist
    return df_dist.rename(columns = name_dict, index = name_dict)

In [8]:
df_ab = get_data(connection,'abilities')
df_in = get_data(connection,'interests')
df_ws = get_data(connection,'work_styles')
df_wv = get_data(connection, 'work_values')
df_wc = get_data(connection,'work_context')
df_knowledge = get_data(connection,'knowledge')
df_gwas = get_data(connection,'work_activities')
df_jz = get_data(connection,'job_zones')
df_jz = df_jz[df_jz['title'] != 'Legislators']
df_skills = get_data(connection,'skills')




In [9]:
wc_dist = distance_table(df_wc)
knowledge_dist = distance_table(df_knowledge)
skills_dist = distance_table(df_skills)
gwas_dist = distance_table(df_gwas)
jz_dist = distance_table(df_jz,True)
wv_dist = distance_table(df_wv)
in_dist = distance_table(df_in)
ab_dist = distance_table(df_ab)
ws_dist = distance_table(df_ws)


In [10]:
skills_dist

,Accountants,Actors,Actuaries,Acupuncturists,Acute Care Nurses,Adapted Physical Education Specialists,Adhesive Bonding Machine Operators and Tenders,"Administrative Law Judges, Adjudicators, and Hearing Officers",Administrative Services Managers,Adult Basic and Secondary Education and Literacy Teachers and Instructors,...,"Welding, Soldering, and Brazing Machine Setters, Operators, and Tenders",Wellhead Pumpers,"Wholesale and Retail Buyers, Except Farm Products",Wind Energy Engineers,Wind Energy Operations Managers,Wind Energy Project Managers,Wind Turbine Service Technicians,"Woodworking Machine Setters, Operators, and Tenders, Except Sawing",Word Processors and Typists,Zoologists and Wildlife Biologists
Accountants,NaN,-0.230680,-1.227414,-1.171535,-0.569831,-1.003623,1.132803,-1.065039,-1.137232,-1.059764,...,0.725944,1.565390,-1.521513,-0.814871,-0.314631,-1.141396,1.107543,1.433590,-0.109065,-0.931834
Actors,-0.230680,NaN,1.135383,-0.864040,0.663371,-0.357262,1.197330,-0.544330,0.050725,-0.943865,...,0.331293,1.241636,0.200386,1.010929,1.070415,0.546771,1.569600,1.321033,-0.680258,0.499286
Actuaries,-1.227414,1.135383,NaN,-0.087157,-0.286412,-0.282347,1.963571,-0.399273,-0.465844,0.038218,...,1.804970,2.486738,-0.972011,-0.930407,-0.086949,-0.831523,1.659458,2.279853,1.216561,-0.704367
Acupuncturists,-1.171535,-0.864040,-0.087157,NaN,-1.001092,-1.412509,0.446884,-0.959553,-0.798526,-1.204994,...,-0.041906,0.854747,-0.723608,-0.542806,-0.172813,-0.692439,0.568397,0.827215,-0.526829,-1.083851
Acute Care Nurses,-0.569831,0.663371,-0.286412,-1.001092,NaN,-1.195340,0.830991,-0.255460,-0.836154,-0.280156,...,0.676692,1.417261,-0.556259,-1.302537,-0.920175,-1.047353,0.542074,1.357017,0.913517,-1.547737
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Wind Energy Project Managers,-1.141396,0.546771,-0.831523,-0.692439,-1.047353,-0.945562,0.908294,-0.497354,-1.831780,-0.280800,...,0.801334,1.456835,-1.831528,-0.911183,-1.592966,NaN,0.644552,1.388393,0.472996,-1.141655
Wind Turbine Service Technicians,1.107543,1.569600,1.659458,0.568397,0.542074,0.979859,-1.277587,1.721333,0.830265,1.343563,...,-0.786879,-0.852132,1.205901,0.564226,-0.197641,0.644552,NaN,-0.962793,0.923107,0.745142
"Woodworking Machine Setters, Operators, and Tenders, Except Sawing",1.433590,1.321033,2.279853,0.827215,1.357017,1.592393,-1.603523,2.089086,1.384270,1.651199,...,-1.709891,-1.733867,1.659570,1.220691,0.909859,1.388393,-0.962793,NaN,0.158726,1.480451
Word Processors and Typists,-0.109065,-0.680258,1.216561,-0.526829,0.913517,0.420100,0.150641,0.492062,0.232212,-0.009116,...,-0.506267,0.315154,0.345638,0.763999,0.925228,0.472996,0.923107,0.158726,NaN,0.631908


In [10]:
df_concat = pd.concat([wc_dist,knowledge_dist,skills_dist,gwas_dist,jz_dist.multiply(1.3)]).groupby(level =0).mean()

### Filters

In [11]:
def filt(df,d):
    '''
    @params: df: Dataframe to filter, d: set of all relevant names
    @returns: filtered Dataframe
    '''
    return df.loc[df.index.isin(d)]

In [12]:
# df_rel = pd.concat([knowledge_dist.multiply(2),skills_dist.multiply(3),gwas_dist]).groupby(level =0).mean()
# df_jz_filt = get_data(connection,'job_zones',True,[2])
df_jz_filt = get_data(connection,'job_zones')
# df_ab_filt = get_data(connection,'abilities',True,'1.A.1.')
df_ab_filt = get_data(connection,'abilities')
# jz_dist_filt = distance_table(df_jz_filt,True)
jz_set = set(df_jz_filt['title'].unique()) #to filter out same job_zones jobs

In [13]:
abilities_filtered = filt(df_ab_filt,jz_set)
skills_filtered = filt(df_skills,jz_set)
knowledge_filtered = filt(df_knowledge,jz_set)
in_filtered = filt(df_in,jz_set)
gwas_filtered = filt(df_gwas,jz_set)

In [14]:
ab_dist_filt = distance_table(abilities_filtered) #NaN?
skills_dist_filt = distance_table(skills_filtered)
knowledge_dist_filt = distance_table(knowledge_filtered)
in_dist_filt = distance_table(in_filtered)
gwas_dist_filt = distance_table(gwas_filtered)
df_concat_filt = pd.concat([ab_dist_filt.multiply(4.0),skills_dist_filt.multiply(3.0),knowledge_dist_filt.multiply(2.0),in_dist_filt,gwas_dist_filt]).groupby(level =0).mean()

## Best Algorithms:

In [15]:
best_algo = pd.concat([skills_dist_filt.multiply(1.3),knowledge_dist_filt.multiply(1.2),in_dist_filt,gwas_dist_filt]).groupby(level =0).mean()

## Validation methods

In [16]:
def percentage_default(occ,df_cps,a,b):
    df_occ = df_cps[df_cps['ONET18_Title_LY'] == occ].sort_values(by = 'pct_socly', ascending = False)
    return df_occ[a:b]['pct_socly'].sum()

In [17]:
def percentage(occ,df_onet,df_cps,a,b):
    '''
    Get total percentage that ONET transition takes ups
    @param occ occupation of interests
    @param df_onet dataFrame for ONET
    @param df_cps dataFrame for CPS data
    @param a index of first position for onet
    @param index of last position
    
    return the percentage composition for the ath to bth place of ONET
    '''
    
    total = 0
    onet_list = df_onet.loc[occ].sort_values()[a:b].index.to_list()
    cps_occ = df_cps[df_cps['ONET18_Title_LY'] == occ]
    for occ in onet_list:
#         print(occ)
#         print(cps_occ[cps_occ['ONET18_Title'] == occ]['pct_tot'].values[0])
        val = cps_occ[cps_occ['ONET18_Title'] == occ]['pct_socly'].values
        if len(val) == 0:
            continue
        else:
            total += val[0]
    
    return total

In [18]:
def percentage_coverage(df_onet,df_cps):
    avg_occ = 0
    count = 0
    occ_max = ''
    max_r = 0
    occ_min = ''
    min_r = 1

    cov_max = ''
    max_c = 0
    cov_min = ''
    min_c = 1
    
    cps_occ = set(df_cps['ONET18_Title_LY'].unique())
    onet_occ = set(df_onet.index.unique())

    relevant_occ = cps_occ & onet_occ
    occ_dictionary = {}
    for occ in relevant_occ:
        cps_len = len(df_cps[df_cps['ONET18_Title_LY'] == occ]['ONET18_Title'])
        if cps_len > 20:
            n = percentage(occ,df_onet,df_cps,0,20)
            d = percentage_default(occ,df_cps,0,20)
            perc = n/d
            count += 1 
            avg_occ += perc
            occ_dictionary[occ] = perc
            if n >= max_c:
                cov_max =occ
                max_c = n 
            if n <= max_c:
                cov_min =occ
                min_c = n  
            if perc >= max_r: 
                occ_max = occ
                max_r = perc
            if perc <= min_r: 
                occ_min = occ
                min_r = perc
    print(100*avg_occ/count)
    print(count)
    print(occ_max, 100*max_r)
    print(occ_min, min_r)
    print(cov_max, max_c)
    print(cov_min, min_c)
    
    return occ_dictionary

In [19]:
def distribution(df):
    df_output = pd.DataFrame.from_dict(df, orient = 'index',  columns= ['score'])
    df_output = df_output.sort_values(by = 'score', ascending = False)
    fig = go.Figure(data=[go.Bar(
                x=df_output.index[:10], y=df_output['score'][:10],
                textposition='auto',
            )])
    fig.update_layout(barmode='stack', xaxis={'categoryorder':'total descending'})

    fig.show()

## algotesting zone

In [27]:
# wc_dist 
# knowledge_dist
# skills_dist 
# gwas_dist 
# jz_dist 
# wv_dist 
# in_dist 
# ab_dist 
# ws_dist

In [28]:
best_algo_flat = pd.concat([skills_dist.multiply(1), #skills 
                       knowledge_dist.multiply(1), #knowledge
                       in_dist.multiply(1), #interest
                       gwas_dist.multiply(1), #general work activities
                       wc_dist.multiply(1), #work context
                       jz_dist.multiply(0), #job zone
                       wv_dist.multiply(0), #work value
                       ab_dist.multiply(1), #abilites
                       ws_dist.multiply(0)] #worksetting
                     ).groupby(level =0).mean()

In [29]:
# best_algo_weighted = pd.concat([skills_dist.multiply(1.2), #skills 
#                        knowledge_dist.multiply(1.3), #knowledge
#                        in_dist.multiply(1), #interest
#                        gwas_dist.multiply(1), #general work activities
#                        wc_dist.multiply(1), #work context
#                        jz_dist.multiply(0), #job zone
#                        wv_dist.multiply(0), #work value
#                        in_dist.multiply(0), #interest
#                        ab_dist.multiply(1), #abilites
#                        ws_dist.multiply(0)] #worksetting
#                      ).groupby(level =0).mean()

In [30]:
# best_algo = pd.concat([skills_dist.multiply(1.2), #skills 
#                        knowledge_dist.multiply(1.3), #knowledge
#                        in_dist.multiply(1), #interest
#                        gwas_dist.multiply(1), #general work activities
#                        wc_dist.multiply(1), #work context
#                        jz_dist.multiply(0), #job zone
#                        wv_dist.multiply(0), #work value
#                        in_dist.multiply(0), #interest
#                        ab_dist.multiply(1.1), #abilites
#                        ws_dist.multiply(0)] #worksetting
#                      ).groupby(level =0).mean()

In [31]:
best_algo

,Accountants,Actors,Actuaries,Acupuncturists,Acute Care Nurses,Adapted Physical Education Specialists,Adhesive Bonding Machine Operators and Tenders,"Administrative Law Judges, Adjudicators, and Hearing Officers",Administrative Services Managers,Adult Basic and Secondary Education and Literacy Teachers and Instructors,...,Wellhead Pumpers,"Wholesale and Retail Buyers, Except Farm Products",Wind Energy Engineers,Wind Energy Operations Managers,Wind Energy Project Managers,Wind Turbine Service Technicians,"Woodworking Machine Setters, Operators, and Tenders, Except Sawing",Word Processors and Typists,Zoologists and Wildlife Biologists,Legislators
Accountants,NaN,0.843973,-1.586641,0.342839,0.566311,-0.126990,0.854312,-0.739770,-1.052051,-0.352737,...,0.753607,-1.458741,0.008241,-0.149954,-0.519015,1.032687,1.067225,-1.078139,-0.155577,0.283850
Actors,0.843973,NaN,1.447812,0.473861,1.141528,-0.108814,1.079083,0.313121,0.771919,-0.639297,...,1.485732,0.459377,1.494273,1.554790,1.043322,1.656607,1.145300,0.216282,1.207360,-0.308203
Actuaries,-1.586641,1.447812,NaN,0.594777,0.601674,0.263861,1.301204,-0.306057,-0.499120,0.238733,...,1.239925,-0.840985,-0.299137,-0.003352,-0.545666,1.323554,1.553095,0.323055,-0.356030,0.272209
Acupuncturists,0.342839,0.473861,0.594777,NaN,-1.744229,-1.315678,0.733134,-0.066782,-0.239334,-0.487198,...,0.640051,0.140409,0.264850,0.340237,0.268521,0.662113,1.241413,0.680535,-0.543701,0.606864
Acute Care Nurses,0.566311,1.141528,0.601674,-1.744229,NaN,-1.446681,0.976733,0.116863,-0.276633,-0.219374,...,0.926109,0.345424,0.118259,0.144855,0.160603,0.789594,1.588767,1.202524,-0.688174,0.427150
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Wind Energy Project Managers,-0.519015,1.043322,-0.545666,0.268521,0.160603,-0.088202,0.874511,-0.246764,-1.654860,0.330278,...,0.609949,-1.357530,-1.321854,-1.702077,NaN,0.173515,1.381730,0.702874,-0.976191,-1.021584
Wind Turbine Service Technicians,1.032687,1.656607,1.323554,0.662113,0.789594,0.523243,-1.457803,1.575832,0.163044,1.339685,...,-1.702230,0.497501,-0.509722,-0.793525,0.173515,NaN,-0.803445,0.872430,0.192037,1.340283
"Woodworking Machine Setters, Operators, and Tenders, Except Sawing",1.067225,1.145300,1.553095,1.241413,1.588767,1.036364,-1.862003,1.602754,1.168105,1.306048,...,-1.751144,0.749945,0.797910,0.914800,1.381730,-0.803445,NaN,0.251795,1.092435,1.694695
Word Processors and Typists,-1.078139,0.216282,0.323055,0.680535,1.202524,0.294953,0.218040,-0.148303,-0.106956,-0.174563,...,0.274083,-0.499798,1.030565,0.767306,0.702874,0.872430,0.251795,NaN,0.815719,0.909749


21.34740997013828

21.17295320510576

### Validating algorithms

In [32]:
best_dict = percentage_coverage(best_algo_flat,df_cps)

20.92052782030208
570
First-Line Supervisors of Non-Retail Sales Workers 89.3305850797751
Extruding and Forming Machine Setters, Operators, and Tenders, Synthetic and Glass Fibers 0.0
Occupational Therapists 79.21193429924357
Food Service Managers 11.853058798236827


In [ ]:
distribution(best_dict)

In [ ]:
df_cps[df_cps['ONET18_Title_LY'] == 'Nurse Practitioners'].sort_values(by = 'pct_socly', ascending = False)[:10][['ONET18_Title','employment_projection_NewJob','wage_change_BetweenJobs','pct_socly']]

In [ ]:
df_cps[df_cps['ONET18_Title_LY'] == 'First-Line Supervisors of Non-Retail Sales Workers'].sort_values(by = 'wage_change_BetweenJobs', ascending = False)[:10][['ONET18_Title','employment_projection_NewJob','wage_change_BetweenJobs','pct_socly']]

In [ ]:
df_cps[df_cps['ONET18_Title_LY'] == 'Retail Salespersons'].sort_values(by = 'pct_socly', ascending = False)[:10][['ONET18_Title','employment_projection_NewJob','wage_change_BetweenJobs','pct_socly']]

In [ ]:
best_algo_weighted.loc['First-Line Supervisors of Non-Retail Sales Workers'].sort_values()[0:10]

In [ ]:
## wages averages

In [ ]:
def goodness_averages(coverage_dict,df_cps,t):
    df_output = pd.DataFrame.from_dict(coverage_dict, orient = 'index',  columns= ['score'])
    df_output = df_output.sort_values(by = 'score', ascending = False)
    
    list_projections = []
    list_wages = []
    for occ in df_output.index:
        df_avg = df_cps[df_cps['ONET18_Title_LY'] == occ].sort_values(by = 'pct_socly', ascending = False)[:10]
        list_projections.append(df_avg["employment_projection_NewJob"].mean())
        list_wages.append(df_avg["wage_change_BetweenJobs"].mean())
        
    df_output.insert(1,"projections_average",list_projections,True)
    df_output.insert(2,"wages_average",list_wages,True)
    fig = go.Figure(data=[go.Bar(
                x=df_output.index[:20], y=df_output[t][:20],
                textposition='auto',
            )])
    fig.update_layout(barmode='stack')

    fig.show()
    
    
    
    return df_output

In [ ]:
d = goodness_averages(best_dict,df_cps, 'wages_average')


In [ ]:
d = goodness_averages(best_dict,df_cps, 'projections_average')